In [1]:
# Dependencies
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup as bs
from sqlalchemy import create_engine

## Extract data

In [2]:
# URL of page to be scraped
url='https://www.kiplinger.com/tool/real-estate/T010-S003-home-prices-in-100-top-u-s-metro-areas/index.php'

# Retrieve page with the requests module
res = requests.get(url)

# Create a Beautiful Soup object
soup = bs(res.content,'lxml')

#  Retrieve the desired table
table = soup.find_all('table')[0] 

# Create a df for the desired data
housePrice_df = pd.read_html(str(table))[0]
housePrice_df.head(10)

,Metro Area,Median Home Price,% Change in 2018,% Change Since Peak*,% Change Since Bottom†,Affordability Index
0,"Akron, Ohio","$135,000",11.3,-9.0,51.8,3
1,"Albany, N.Y.",182000,8.7,5.0,18.5,8
2,"Albuquerque, N.M.",167000,3.2,7.6,28.8,7
3,"Allentown, Pa.",175000,11.8,-10.2,32.4,5
4,"Atlanta, Ga.",210000,8.9,5.8,106.4,4
5,"Augusta, Ga.",155000,5.7,6.8,25.3,1
6,"Austin, Texas",290000,2.2,81.1,80.0,9
7,"Bakersfield, Calif.",215000,6.8,-21.3,81.5,7
8,"Baltimore, Md.",248000,6.0,-15.8,26.7,5
9,"Baton Rouge, La.",168000,1.7,31.6,14.4,3


In [3]:
# List the names of all columns
housePrice_df.columns

Index(['Metro Area', 'Median  Home Price', '% Change  in 2018',
       '% Change  Since Peak*', '% Change  Since Bottom†',
       'Affordability Index'],
      dtype='object')

## Transform data

In [4]:
# Create new data with select columns
df_region_prices = housePrice_df[["Metro Area","Median  Home Price"]].copy()
df_region_prices.head()

,Metro Area,Median Home Price
0,"Akron, Ohio","$135,000"
1,"Albany, N.Y.",182000
2,"Albuquerque, N.M.",167000
3,"Allentown, Pa.",175000
4,"Atlanta, Ga.",210000


In [5]:
#  Split Metro Area in city/state and select only the City
#df_region_prices['Metro Area'].str.split(',',1, expand=True)

df_region_prices["Metro Area"] = df_region_prices['Metro Area'].str.split(',', n=1, expand=True)[0]

In [6]:
df_region_prices.head(100)

,Metro Area,Median Home Price
0,Akron,"$135,000"
1,Albany,182000
2,Albuquerque,167000
3,Allentown,175000
4,Atlanta,210000
5,Augusta,155000
6,Austin,290000
7,Bakersfield,215000
8,Baltimore,248000
9,Baton Rouge,168000


In [7]:
# Rename columns
df_region_prices.rename(columns={'Metro Area':'Metro City','Median  Home Price':'Median Home Price'}, inplace=True)
df_region_prices.head(10)

,Metro City,Median Home Price
0,Akron,"$135,000"
1,Albany,182000
2,Albuquerque,167000
3,Allentown,175000
4,Atlanta,210000
5,Augusta,155000
6,Austin,290000
7,Bakersfield,215000
8,Baltimore,248000
9,Baton Rouge,168000


In [8]:
# Remove "$" sign from the first record
df_region_prices['Median Home Price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [9]:
df_region_prices.head(10)

,Metro City,Median Home Price
0,Akron,135000
1,Albany,182000
2,Albuquerque,167000
3,Allentown,175000
4,Atlanta,210000
5,Augusta,155000
6,Austin,290000
7,Bakersfield,215000
8,Baltimore,248000
9,Baton Rouge,168000


In [10]:
# Export dataframe to a csv file
df_region_prices.to_csv('housePrices_bs.csv', index=False)

## Load: Connect to local database

In [15]:
rds_connection_string = "postgres:postgres@localhost:5432/avocado_vs_house"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [16]:
engine.table_names()

['avocado_db', 'houseprices_db']